In [43]:
import collections
import json
import re

import xlrd


In [164]:
class Dict(dict):
    def __getattr__(self, key):
        return self[key]

def extract(fn):
    data = Dict()
    xls = xlrd.open_workbook(fn)
    for sheet in xls.sheets():
        keys = sheet.row_values(0)
        assert all([isinstance(k, str) for k in keys])
        data[sheet.name] = [Dict(zip(keys, sheet.row_values(idx))) for idx in range(1, sheet.nrows)]
    return data
    

class _Env(dict):
    def __missing__(self, key):
        return key
env = _Env()

In [52]:
week_days = set(map("周{}".format, range(1, 8)))
week_days

{'周1', '周2', '周3', '周4', '周5', '周6', '周7'}

In [165]:
xls = extract("test.xls")

In [166]:
def convert_row_to_resources(row):
    row = row.copy()
    name = row.pop("姓名")
    up = row.pop("师父")
    l = []
    times = []
    for d in week_days:
        times.extend((d, i.strip()) for i in row.pop(d).split(","))
    for d, t in times:
        if not t:
            continue
        for skill, n in row.items():
            if not n:
                continue
            l.append(
                (
                    (d, t, skill),
                    {"name": name, "point": float(n)}
                )
            )
    return l
            
        
def generate_resources(xls):
    producers = collections.defaultdict(list)
    for name, rows in xls.items():
        if name.startswith("人员"):
            for row in rows:
                for task, resource in convert_row_to_resources(row):
                    producers[task].append(resource)
#                 print(x(row))
    return producers
                
resources = generate_resources(xls)
resources

defaultdict(list,
            {('周1', '上午', '心脏'): [{'name': '许嵩', 'point': 3.0},
              {'name': '老王', 'point': 3.0}],
             ('周1', '上午', '普通'): [{'name': '许嵩', 'point': 3.0},
              {'name': '老王', 'point': 3.0}],
             ('周1', '上午', '菊花'): [{'name': '老王', 'point': 5.0}],
             ('周1', '下午', '心脏'): [{'name': '老王', 'point': 3.0}],
             ('周1', '下午', '普通'): [{'name': '老王', 'point': 3.0}],
             ('周1', '下午', '菊花'): [{'name': '老王', 'point': 5.0}],
             ('周1', '全天', '心脏'): [{'name': '朱冶文', 'point': 1.0}],
             ('周1', '全天', '普通'): [{'name': '朱冶文', 'point': 1.0},
              {'name': '周杰伦', 'point': 1.0}],
             ('周1', '全天', '菊花'): [{'name': '朱冶文', 'point': 1.0}],
             ('周1', '没空', '心脏'): [{'name': '方文山', 'point': 1.0}],
             ('周1', '没空', '普通'): [{'name': '方文山', 'point': 1.0}],
             ('周2', '上午', '心脏'): [{'name': '许嵩', 'point': 3.0},
              {'name': '老王', 'point': 3.0}],
             ('周2', 

In [167]:
def convert_row_to_requirements(day, row):
    row = row.copy()
    l = []
    
    for k, v in row.items():
        if "/" not in k:
            continue
        if not v:
            continue
        t, skill = k.split("/")
        l.append({
            "area": row["区域"],
            "target": row["任务"],
            "require": (day, t, skill),
            "cost": float(v),
        })
        
    return l
    
        
def generate_requirements(xls):
    requirements = []
    for day, rows in xls.items():
        if day in week_days:
            for row in rows:
                for r in convert_row_to_requirements(day, row):
                    requirements.append(r)
                    
    def key(x):
        day, t, skill = x["require"]
        return skill == "普通", day, t
    
    requirements.sort(key=key)
    return requirements
    
requirements = generate_requirements(xls)
requirements

[{'area': '2区', 'cost': 2.0, 'require': ('周1', '上午', '心脏'), 'target': '18室'},
 {'area': '2区', 'cost': 1.0, 'require': ('周1', '下午', '心脏'), 'target': '18室'},
 {'area': '2区', 'cost': 1.0, 'require': ('周1', '全天', '心脏'), 'target': '18室'},
 {'area': '2区', 'cost': 2.0, 'require': ('周2', '上午', '心脏'), 'target': '18室'},
 {'area': '2区', 'cost': 1.0, 'require': ('周2', '下午', '心脏'), 'target': '18室'},
 {'area': '2区', 'cost': 1.0, 'require': ('周2', '全天', '心脏'), 'target': '18室'},
 {'area': '1区', 'cost': 1.0, 'require': ('周1', '上午', '普通'), 'target': '1室'},
 {'area': '1区', 'cost': 1.0, 'require': ('周1', '上午', '普通'), 'target': '2室'},
 {'area': '1区', 'cost': 1.0, 'require': ('周1', '上午', '普通'), 'target': '3室'},
 {'area': '1区', 'cost': 1.0, 'require': ('周1', '上午', '普通'), 'target': '4室'},
 {'area': '1区', 'cost': 1.0, 'require': ('周1', '上午', '普通'), 'target': '5室'},
 {'area': '1区', 'cost': 1.0, 'require': ('周1', '上午', '普通'), 'target': '6室'},
 {'area': '1区', 'cost': 1.0, 'require': ('周1', '上午', '普通'), 'target': 

In [163]:

resources = generate_resources(xls)




def plan():
    empty = []
    accumulated = collections.Counter()
    lst = []
    areas_map = {}
    
    for item in requirements:
        require = item["require"]
        cost = item["cost"]
        target = item["target"]
        area = item["area"]
        reses = resources.get(require, empty)

        for res in reses:
            name, point = res["name"], res["point"]
            flag = (name, require[0], require[1])
            if point >= cost and areas_map.setdefault(flag, area) == area:
                res["point"] -= cost
                accumulated[name] += 1
                break
        else:
            name = "----"
            
        lst.append((name, target, require))
        reses.sort(key=lambda x: accumulated[x["name"]])

        print(name, target, *require, sep="\t")#, cost, res["point"])

    print(accumulated, areas_map)
    return lst
    

p = plan()
p.sort(key=lambda x: (x[1], x[0]))
p.sort()
p

许嵩	18室	周1	上午	心脏
老王	18室	周1	下午	心脏
朱冶文	18室	周1	全天	心脏
许嵩	18室	周2	上午	心脏
老王	18室	周2	下午	心脏
朱冶文	18室	周2	全天	心脏
老王	1室	周1	上午	普通
老王	2室	周1	上午	普通
老王	3室	周1	上午	普通
----	4室	周1	上午	普通
----	5室	周1	上午	普通
----	6室	周1	上午	普通
----	7室	周1	上午	普通
----	8室	周1	上午	普通
----	9室	周1	上午	普通
----	10室	周1	上午	普通
----	11室	周1	上午	普通
许嵩	12室	周1	上午	普通
许嵩	13室	周1	上午	普通
许嵩	14室	周1	上午	普通
----	15室	周1	上午	普通
----	16室	周1	上午	普通
----	17室	周1	上午	普通
----	1室	周1	下午	普通
----	2室	周1	下午	普通
----	3室	周1	下午	普通
----	4室	周1	下午	普通
----	5室	周1	下午	普通
----	6室	周1	下午	普通
----	7室	周1	下午	普通
----	8室	周1	下午	普通
----	9室	周1	下午	普通
----	10室	周1	下午	普通
----	11室	周1	下午	普通
老王	12室	周1	下午	普通
老王	13室	周1	下午	普通
老王	14室	周1	下午	普通
----	15室	周1	下午	普通
----	16室	周1	下午	普通
----	17室	周1	下午	普通
周杰伦	11室	周1	全天	普通
朱冶文	12室	周1	全天	普通
----	13室	周1	全天	普通
----	14室	周1	全天	普通
----	15室	周1	全天	普通
----	16室	周1	全天	普通
----	17室	周1	全天	普通
老王	1室	周2	上午	普通
老王	2室	周2	上午	普通
老王	3室	周2	上午	普通
----	4室	周2	上午	普通
----	5室	周2	上午	普通
----	6室	周2	上午	普通
----	7室	周2	上午	普通
----	8室	周2	上午	普通
----	9室	周2	上午	普通
----	10室	周2	上午	普通
----	11室	周2	上午	普通
许嵩	12室	周2	上午	普通
许嵩	

[('----', '10室', ('周1', '上午', '普通')),
 ('----', '10室', ('周1', '下午', '普通')),
 ('----', '10室', ('周2', '上午', '普通')),
 ('----', '10室', ('周2', '下午', '普通')),
 ('----', '11室', ('周1', '上午', '普通')),
 ('----', '11室', ('周1', '下午', '普通')),
 ('----', '11室', ('周2', '上午', '普通')),
 ('----', '11室', ('周2', '下午', '普通')),
 ('----', '13室', ('周1', '全天', '普通')),
 ('----', '14室', ('周1', '全天', '普通')),
 ('----', '14室', ('周2', '全天', '普通')),
 ('----', '15室', ('周1', '上午', '普通')),
 ('----', '15室', ('周1', '下午', '普通')),
 ('----', '15室', ('周1', '全天', '普通')),
 ('----', '15室', ('周2', '上午', '普通')),
 ('----', '15室', ('周2', '下午', '普通')),
 ('----', '15室', ('周2', '全天', '普通')),
 ('----', '16室', ('周1', '上午', '普通')),
 ('----', '16室', ('周1', '下午', '普通')),
 ('----', '16室', ('周1', '全天', '普通')),
 ('----', '16室', ('周2', '上午', '普通')),
 ('----', '16室', ('周2', '下午', '普通')),
 ('----', '16室', ('周2', '全天', '普通')),
 ('----', '17室', ('周1', '上午', '普通')),
 ('----', '17室', ('周1', '下午', '普通')),
 ('----', '17室', ('周1', '全天', '普通')),
 ('----', '1